## 사이킷런을 사용하지 않고 넘파이만 사용하여 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요.

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

raw_data = load_iris()
raw_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [2]:
raw_data['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
X = raw_data['data'][:, (2,3)]
y = raw_data['target']
X[:10]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

**편향 추가**

입력값이 다 0일 때 활성화 함수가 제대로 동작하지 않을 수 있음. 예를들어 정규화한 x의 값이 0이라면 편향이 없는 모델에 들어갔을 때 출력값이 0이 나와 실제 값이 유의미함에도 불구하고 ReLU와 같은 활성화함수에서는 비활성화되어 학습이 제대로 이루어지지 않을 수도 있다. 따라서 편향인 b를 추가하여 값이 0이 되지 않도록 방지하고 더욱 유연하게 학습이 될 수 있도록 보장한다.

In [4]:
biased_X = np.c_[np.ones(len(X)), X]
biased_X[:5]

array([[1. , 1.4, 0.2],
       [1. , 1.4, 0.2],
       [1. , 1.3, 0.2],
       [1. , 1.5, 0.2],
       [1. , 1.4, 0.2]])

# 훈련, 검증, 테스트 셋으로 나누기

In [5]:
# 비율조정
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(biased_X)

test_size = int(total_size*test_ratio)
validation_size = int(total_size*test_ratio)
train_size = total_size - test_size - validation_size

print(total_size)
print(train_size, validation_size, test_size)

150
90 30 30


In [16]:
# 실제 데이터 나눠담기
np.random.seed(42)

shuffle = np.random.permutation(total_size)

train_X = biased_X[shuffle[:train_size]]
train_y = y[shuffle[:train_size]]
val_X = biased_X[shuffle[train_size:-test_size]]
val_y = y[shuffle[train_size:-test_size]]
test_X = biased_X[shuffle[-test_size:]]
test_y = y[shuffle[-test_size:]]

print(train_X.shape)
print(val_X.shape)
print(test_X.shape)

(90, 3)
(30, 3)
(30, 3)


# 소프트맥스 회귀를 위한 타겟 값 조정

현재 y값은 0, 1, 2 세 값중 하나로 돼있음. 이를 0, 1, 2 각각에 대한 확률값으로 표현되도록 바꾸어주어야 됨. 예를들어 클래스가 0인 y 값을 클래스가 0일 확률 1, 1일 확률 0, 2일 확률 0과 같이 바꿔줘야 됨.

In [17]:
def to_one_hot(y):
  n_classes = y.max() + 1
  Y_one_hot = np.zeros((len(y), n_classes))
  Y_one_hot[np.arange(len(y)), y] = 1
  return Y_one_hot

to_one_hot(train_y[:10])

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [18]:
train_y_oh = to_one_hot(train_y)
test_y_oh = to_one_hot(test_y)
val_y_oh = to_one_hot(val_y)

<img src = "https://blog.kakaocdn.net/dn/bLXHTX/btq0JxqeTxt/4L43VcTsdKiPOuFJABg7p0/img.png" height = 150 width = 300>

In [43]:
# 소프트맥스 함수 구현
def softmax(logits):
    exp = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # 오버플로우 방지
    exp_sum = np.sum(exp, axis=1, keepdims=True)
    return exp / exp_sum

n_inputs = train_X.shape[1]
n_outputs = train_y_oh.shape[1]

eta = 0.01
n_iterations = 5000
theta = np.random.randn(n_inputs, n_outputs)

best_loss = np.infty
stagnant_epochs = 0
patience = 1  # 조기 종료 기준 5

for iteration in range(n_iterations):
    logits = train_X.dot(theta)
    y_proba = softmax(logits)
    error = y_proba - train_y_oh
    gradients = (1 / len(train_X)) * train_X.T.dot(error)
    theta -= eta * gradients

    logits = val_X.dot(theta)
    y_proba = softmax(logits)
    loss = -np.mean(np.sum(val_y_oh * np.log(y_proba + 1e-8), axis=1))

    if loss < best_loss:
        best_loss = loss
        stagnant_epochs = 0  # 개선되었으므로 초기화
    else:
        stagnant_epochs += 1  # 개선되지 않으면 증가

    if stagnant_epochs >= patience:
        print(iteration, loss, "조기종료수행!")
        break

    if iteration % 500 == 0:
        print(iteration, loss)


0 4.0067861708776915
500 0.8251042681737477
1000 0.6835348864935484
1500 0.6032365946279499
2000 0.5517449170307135
2500 0.5153322379523158
3000 0.48776748809294346
3500 0.4658959247946664
4000 0.44795230143831655
4500 0.432863973439168


In [46]:
logits = val_X.dot(theta)
prob_y = softmax(logits)
pred_y = np.argmax(prob_y, axis=1) # np.argmax: 가장 높은 값의 인덱스 반환
accuracy = np.mean(pred_y == val_y)
print(accuracy)

0.9333333333333333
